# SPARQL Queries - Encyclopaedia Britannica - EB-KG

This notebook enables to perform several SPARQL queries to the EB-KG Knowlege Graph (total_eb.ttl) 
It includes two types RDF queries:

 - Querying the KG locally: Using rdflib.plugins.sparql
 - Querying the KG from an Apache Fuseki Server: Using SPARQLWrapper
  


The EB-KG implements the [EB-Ontology](https://francesnlp.github.io/EB-ontology/doc/index-en.html), which has the following [Data Model](https://francesnlp.github.io/EB-ontology/doc/dataModel.png)

The EB-KG (total_eb.ttl) can be download from [Zenodo](https://zenodo.org/record/6673897#.YrHZ85BBx_A)

In general terms this EB-Ontology can be described as follows:

1. The Encyclopaedia has several Editions
2. An Edition has several Volumes (Books)
3. An Edition can have several Editors, Publishers, and reference books
3. A Volume can have several Terms.
4. A Term can be either an Article or Topic
5. A Term has a name (term name , e.g. Abacus) and a definition ( e.g. definition text of Abacus). 
6. A Volume can have several Pages. 
7. In the Page we haven’t included the text. 
8. A Term starts in a Page
9. A Term ends in a Page
10. A Supplement can have several Editors, Publishers and reference books
11. A Supplement can have several Volumes.

Example of values of the EB-KG properties:


    mmsid: 992277653804341
    edition title: First edition, 1771, Volume 1, A-B
    editor name: Smellie, William
    editor date: 1740-1795
    genre: encyclopedia
    language: eng
    termsOfAddress: NaN
    numberOfPages: 832
    physicalDescription: 3 v., 160 plates : ill. ; 26 cm. (4to)
    place: Edinburgh
    publisher: Printed for A. Bell and C. Macfarquhar; and so...
    referencedBy: [Alston, R.C. Engl. language III, 560, ESTC T...
    shelfLocator: EB.1
    editionSubTitle: Illustrated with one hundred and sixty copperp...
    volume title: Encyclopaedia Britannica; or, A dictionary of ...
    year: 1771
    volumeId: 144133901
    metsXML: 144133901-mets.xml
    permanentURL: https://digital.nls.uk/144133901
    publisherPersons: [C. Macfarquhar, Colin Macfarquhar]
    volume number: 1
    letters: A-B
    part: 0
    edition number : 1
    supplement title:
    supplement subtitle:
    supplementsTo: []
    numberOfVolumes: 6
    term: OR
    definition: A NEW A D I C T I A A, the name of several riv...
    relatedTerms: []
    header: EncyclopaediaBritannica
    startsAt: 15
    endsAt: 15
    numberOfTerms: 22
    numberOfWords: 54
    positionPage: 0
    typeTerm: Article
    altoXML: 144133901/alto/188082904.34.xml




### Loading the necessary libraries

In [7]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as pl
from rdflib import Graph, ConjunctiveGraph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

In [8]:
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [9]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

### Functions

In [10]:
def plot_resource(results):

    G = rdflib_to_networkx_multidigraph(results)
    pos = nx.spring_layout(G, scale=3)
    edge_labels = nx.get_edge_attributes(G, 'r')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    nx.draw(G, with_labels=True)
    plt.show()

### Type 1: Loading the graph locally

In [11]:
g = Graph()
#modify the path if necessary
g.parse("../../results_NLS/total_eb.ttl", format="ttl") 

<Graph identifier=N4dea341c291945919778af204f10c9c0 (<class 'rdflib.graph.Graph'>)>

Query 1: List all the resources with the property eb:editor

In [13]:
eb = Namespace("https://w3id.org/eb#")

q1 = prepareQuery('''
  SELECT ?Edition WHERE { 
    ?Edition eb:editor ?FullName. 
  }
  ''',
  initNs = { "eb": eb}
)


for r in g.query(q1):
      print(r.Edition)

https://w3id.org/eb/i/Edition/9910796233804340
https://w3id.org/eb/i/Edition/9910796273804340
https://w3id.org/eb/i/Edition/9929777383804340
https://w3id.org/eb/i/Edition/9910796343804340
https://w3id.org/eb/i/Edition/9929192893804340
https://w3id.org/eb/i/Edition/992277653804341


Query 2: Same query but asking more information about the resources obtained. 

In [14]:
q2 = prepareQuery('''
  SELECT ?Subject ?Editor WHERE { 
    ?Subject eb:editor ?Editor.
  } 
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q2):
  print(r.Subject, r.Editor)

https://w3id.org/eb/i/Edition/9910796233804340 https://w3id.org/eb/i/Person/Millar,James
https://w3id.org/eb/i/Edition/9910796273804340 https://w3id.org/eb/i/Person/Stewart,Dugald
https://w3id.org/eb/i/Edition/9929777383804340 https://w3id.org/eb/i/Person/Stewart,Dugald
https://w3id.org/eb/i/Edition/9910796343804340 https://w3id.org/eb/i/Person/Gleig,George
https://w3id.org/eb/i/Edition/9929192893804340 https://w3id.org/eb/i/Person/Smellie,William
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb/i/Person/Smellie,William


Query 3: Obtaining just the editors names 

In [15]:
q3 = prepareQuery('''
SELECT DISTINCT ?name
    WHERE {
     ?instance eb:editor ?Editor.
     ?Editor eb:name ?name .
    }
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q3):
      print(r.name)

Gleig, George
Millar, James
Smellie, William
Stewart, Dugald


In [16]:
res=g.query(q3)
a=list(res)[0]
a.name

rdflib.term.Literal('Gleig, George', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))

Query 4: Same query asking for the first 10 resources with the properity eb.name

In [17]:
q4 = prepareQuery('''
  SELECT ?Subject ?Name WHERE { 
    ?Subject eb:name ?Name.
  } 
  LIMIT 10
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q4):
    print(r.Subject, r.Name)

https://w3id.org/eb/i/Person/C.Elliot C. Elliot
https://w3id.org/eb/i/Person/ColinMacfarquhar Colin Macfarquhar
https://w3id.org/eb/i/Person/Gleig,George Gleig, George
https://w3id.org/eb/i/Person/J.Bell J. Bell
https://w3id.org/eb/i/Person/J.Brown J. Brown
https://w3id.org/eb/i/Person/J.Dickson J. Dickson
https://w3id.org/eb/i/Person/JohnDonaldson John Donaldson
https://w3id.org/eb/i/Person/Millar,James Millar, James
https://w3id.org/eb/i/Person/W.Gordon W. Gordon
https://w3id.org/eb/i/Article/9910796233804340_191253819_AGRIFOLIUM_0 AGRIFOLIUM


Query 5: Obtain the resources titles which have been printed in London - limiting to 10 results

In [19]:
from rdflib import XSD
q5 = prepareQuery('''
SELECT DISTINCT ?name
WHERE {
 ?instance eb:editor ?editor;
           eb:place ?place;
           eb:title ?name.
  
}
LIMIT 10
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q5, initBindings = {'?place' : Literal('London', datatype=XSD.string)}):
    print(r.name)

Query 6: Asking for resources which name is "Smellie, Willian"

In [13]:
from rdflib import XSD
q6 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject eb:name ?Family.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q6, initBindings = {'?Family' : Literal('Smellie, William', datatype=XSD.string)}):
  print(r.Subject)

https://w3id.org/eb/i/Person/Smellie,William


Query 7: Asking for resources uris which has ABACUS in their term names

In [20]:
from rdflib import XSD
q7 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject eb:name ?Term.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q7, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject)

https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_0
https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_1
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_0
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_1
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_2
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_0
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_1
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_2
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_0
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_2
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_3
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_0
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_2
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_3
https://w3id.org/eb/i/Article/997902523804341_144850370_ABACUS_0
https://w3id.org/

Query 8: Asking for resources uris and terms names which has ABACUS in their term names

In [21]:
from rdflib import XSD
q8 = prepareQuery('''
  SELECT ?Subject ?Term WHERE { 
    ?Subject eb:name ?Term.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q8, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.Term)

https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_0 ABACUS
https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_1 ABACUS
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_0 ABACUS
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_1 ABACUS
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_2 ABACUS
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_0 ABACUS
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_1 ABACUS
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_2 ABACUS
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_0 ABACUS
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_2 ABACUS
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_3 ABACUS
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_0 ABACUS
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_2 ABACUS
https://w3id.org/eb/i/Article/9929192893804340_144850366_

Query 9: Asking for all Edition resources

In [22]:
q9 = prepareQuery('''
    SELECT ?edition
    WHERE {
       ?edition rdf:type eb:Edition .
    }
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q9):
  print(r.edition)

https://w3id.org/eb/i/Edition/9910796233804340
https://w3id.org/eb/i/Edition/9910796253804340
https://w3id.org/eb/i/Edition/9910796273804340
https://w3id.org/eb/i/Edition/9910796343804340
https://w3id.org/eb/i/Edition/9922270543804340
https://w3id.org/eb/i/Edition/9929192893804340
https://w3id.org/eb/i/Edition/9929777383804340
https://w3id.org/eb/i/Edition/997902523804341
https://w3id.org/eb/i/Edition/997902543804341
https://w3id.org/eb/i/Edition/992277653804341


Query 10: Asking for the first 10 topics resources

In [23]:
q10 = prepareQuery('''
    SELECT ?topic
    WHERE {
       ?topic rdf:type eb:Topic .
    }
    LIMIT 10
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q10):
  print(r.topic)

https://w3id.org/eb/i/Topic/9910796233804340_191253819_JTXVF_0
https://w3id.org/eb/i/Topic/9910796233804340_191253825_J_0
https://w3id.org/eb/i/Topic/9910796233804340_191253838_GUZZERAT_0
https://w3id.org/eb/i/Topic/9910796233804340_191253838_MICKOSCOK_0
https://w3id.org/eb/i/Topic/9910796233804340_191253838_SPMMPP_0
https://w3id.org/eb/i/Topic/9910796233804340_191320558_IROJKOTIONEHEE_0
https://w3id.org/eb/i/Topic/9910796233804340_191320558_SIII_0
https://w3id.org/eb/i/Topic/9910796233804340_191678900_SUHGERYPDXX_0
https://w3id.org/eb/i/Topic/9910796233804340_191678901_WPJCPDXIA_0
https://w3id.org/eb/i/Topic/9910796233804340_191678902_MECH_0


Query 11: Asking for the first 5 articles resources, with their related terms, names, and definitions

In [28]:
q11 = prepareQuery('''
    SELECT *
    WHERE {
       ?article a eb:Article .
       ?article eb:relatedTerms ?relatedTerms . 
       ?article eb:name ?name .
       ?article eb:definition ?definition.
    }
    LIMIT 5
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q11):
  print(r.article, r.relatedTerms, r.name, r.definition)
  print("----")

https://w3id.org/eb/i/Article/9910796233804340_191253819_AGROM_0 https://w3id.org/eb/i/Article/9910796233804340_191678901_INDEX_0 AGROM a disease frequent in Bengal parts of the Indies, in wmich the tongue chaps, and is sometimes covered with white Indians are very fearful of this disease, which tribute to extreme heat of the stomach. is, to drink some chalybeate liquor, or the AGROSTEMMA, Wild Lychnis, in Botany. See Botany Index.
----
https://w3id.org/eb/i/Article/9910796233804340_191253819_AGROM_0 https://w3id.org/eb/i/Article/9910796233804340_191679019_INDEX_0 AGROM a disease frequent in Bengal parts of the Indies, in wmich the tongue chaps, and is sometimes covered with white Indians are very fearful of this disease, which tribute to extreme heat of the stomach. is, to drink some chalybeate liquor, or the AGROSTEMMA, Wild Lychnis, in Botany. See Botany Index.
----
https://w3id.org/eb/i/Article/9910796233804340_191253819_AGUE_0 https://w3id.org/eb/i/Article/9910796233804340_1916789

Query 12:  Asking for the terms which have ABACUS in their names, along with their definitions, Volume URI, and Edition Title

In [29]:
q12 = prepareQuery('''
  SELECT  ?Subject ?Definition ?HasPart ?Title WHERE {
    ?Subject eb:name ?Term.
    ?Subject eb:definition ?Definition.
    ?HasPart eb:hasPart ?Subject.
    ?HasPart eb:title ?Title
  }  
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q12, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.Definition, r.HasPart, r.Title)
  print("----")



https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_0 among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says they had their abaci, beds, &c. plated over with gold. https://w3id.org/eb/i/Volume/9929777383804340_193322698 Encyclopaedia Britannica
----
https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACUS_1 or https://w3id.org/eb/i/Volume/9929777383804340_193322698 Encyclopaedia Britannica
----
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_0 among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says, They had their abaci, beds, &c. plated https://w3id.org/eb/i/Volume/997902543804341_190273291 Encyclopaedia Britannica
----
https://w3id.org/eb/i/Article/997902543804341_190273291_ABACUS_1 Major, in metallurgic operations, the name of a trough used in the mines, wherein the

In [30]:
q13 = prepareQuery('''
  SELECT  ?Subject ?RelatedTerms WHERE {
    ?Subject eb:name ?Term.
    ?RelatedTerms eb:relatedTerms ?Subject.

    
  }  
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q13, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.RelatedTerms)
  print("----")


https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_0 https://w3id.org/eb/i/Article/9910796253804340_192200322_CHINESE_0
----
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_1 https://w3id.org/eb/i/Article/9910796253804340_192200322_CHINESE_0
----
https://w3id.org/eb/i/Article/9910796253804340_192547781_ABACUS_2 https://w3id.org/eb/i/Article/9910796253804340_192200322_CHINESE_0
----
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_0 https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0
----
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_2 https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0
----
https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_3 https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0
----
https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_0 https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACISCUS_0
----
https://w3id.org/eb/i/Article/99

In [23]:
#G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
#pos = nx.spring_layout(G, scale=2)
#edge_labels = nx.get_edge_attributes(G, 'r')
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#nx.draw(G, with_labels=True)

#if not in interactive mode for 
#plt.show()


### Type 2: Connecting with FUSEKI and using SPARQLWrapper

**Previously you need to have uploaded the Knowlege Graph (total_eb.ttl) into Apache Fuseki**

Query 14: Basic query

In [31]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
    SELECT ?subject ?predicate ?object WHERE {   ?subject ?predicate ?object } LIMIT 5 
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results

for result in results["results"]["bindings"]:
    print(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"] )


https://w3id.org/eb/i/Article/9910796253804340_192200323_MATRICES_0 https://w3id.org/eb#position 5
https://w3id.org/eb/i/Article/9910796253804340_192200323_MATRICES_0 https://w3id.org/eb#numberOfWords 57
https://w3id.org/eb/i/Article/9910796253804340_192200323_MATRICES_0 https://w3id.org/eb#name MATRICES
https://w3id.org/eb/i/Article/9910796253804340_192200323_MATRICES_0 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Article
https://w3id.org/eb/i/Article/9910796253804340_192200323_MATRICES_0 https://w3id.org/eb#definition used in coining, are pieces of steel in form of dies, whereon are engraven the several figures, arms, characters, legends, &c. wherewith the species are to be stamped. The engraving is performed with several puncheons, which being formed in relievo, or prominent, when struck on the metal, make an indented impression, which the French call en creux.


Query 15: Describe query

In [33]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
    DESCRIBE <https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0>
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [34]:
for s,p,o in results:
      print(s,p,o)

https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#name PAISLEY
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Article
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#position 4
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#numberOfWords 14
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#startsAtPage https://w3id.org/eb/i/Page/9929192893804340_144850368_526
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#endsAtPage https://w3id.org/eb/i/Page/9929192893804340_144850368_526
https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0 https://w3id.org/eb#definition a town of Scotland, in the county of Renfrew, six miles west of Glasgow.


Query 16: Print definition, article, start page number, end page number, year, volume number, edition number, related terms, permanent URL of Articles resources that have ABACUS in their term name. 

In [35]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
term="ABACUS"
query="""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?definition ?article ?spnum ?epnum ?year ?vnum ?enum ?rn ?permanentURL WHERE {
       ?article a eb:Article ;
                eb:name "%s" ;
                eb:definition ?definition ;
        OPTIONAL {?article eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
       ?e eb:hasPart ?v.
       ?v eb:number ?vnum.
       ?v eb:permanentURL ?permanentURL.
       ?v eb:hasPart ?article.
       ?e eb:publicationYear ?year.
       ?e eb:number ?enum.
       ?article eb:startsAtPage ?sp.
       ?sp   eb:number ?spnum .
       ?article eb:endsAtPage ?ep.
       ?ep eb:number ?epnum .
               
       }
    """ % (term)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r)
    break 

{'definition': {'type': 'literal', 'value': 'among the ancients, was a kind of cup Livy, deferibing the luxury into which the Romans degenerated after the conquest of Alia, says they had their abaci, beds, &c. plated over'}, 'article': {'type': 'uri', 'value': 'https://w3id.org/eb/i/Article/9922270543804340_192016229_ABACUS_0'}, 'spnum': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '22'}, 'epnum': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '22'}, 'year': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1815'}, 'vnum': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1'}, 'enum': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '5'}, 'permanentURL': {'type': 'literal', 'value': 'https://digital.nls.uk/192016229'}}


Query 17: Print definition, article, start page number, end page number, year, volume number, edition number, related terms, permanent URL of Articles or Topics resources that have ABACUS in their term name. 

In [36]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
term="ABACUS"
query="""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?definition ?a ?b  ?spnum ?epnum 
    WHERE {{
    	?b a eb:Article .
    	?b eb:name ?a .
        ?b eb:name "%s" .
        ?b eb:definition ?definition . 
        OPTIONAL {?b eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
       ?e eb:hasPart ?v.
       ?v eb:number ?vnum.
       ?v eb:permanentURL ?permanentURL.
       ?v eb:hasPart ?b.
       ?e eb:publicationYear ?year.
       ?e eb:number ?enum.
       ?b eb:startsAtPage ?sp.
       ?sp   eb:number ?spnum .
       ?b eb:endsAtPage ?ep.
       ?ep eb:number ?epnum .
        
        }
  		UNION {
    	?b a eb:Topic .
    	?b eb:name ?a .
        ?b eb:name "%s" .
        ?b eb:definition ?definition 
        
        OPTIONAL {?b eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
        
        ?e eb:hasPart ?v.
        ?v eb:number ?vnum.
        ?v eb:permanentURL ?permanentURL.
        ?v eb:hasPart ?b.
        ?e eb:publicationYear ?year.
        ?e eb:number ?enum.
        ?b eb:startsAtPage ?sp.
        ?sp   eb:number ?spnum .
        ?b eb:endsAtPage ?ep.
        ?ep eb:number ?epnum .
        
        }
   }
""" %(term, term)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["a"]["value"])
    print(r["definition"]["value"])

ABACUS
among the ancients, was a kind of cup Livy, deferibing the luxury into which the Romans degenerated after the conquest of Alia, says they had their abaci, beds, &c. plated over
ABACUS
among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says they had their abaci, beds, &c. plated over with gold.
ABACUS
among the ancients, was a kind of cupboard, or buffet. J Livy, deferibing the luxury into which the Romans degenerated after the conquest of Asia, says, They had their abaci, beds, &c. plated over with gold. (Dec. IV. Lib. ix.)
ABACUS
in architecture, II. 5; III. 464 —cupboard, II. 5—arithmetical instrument, ib.
ABACUS
among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says they had their abaci, beds, &c. plated over
ABACUS
Major, in metallurgic operations, the name of a trough used in the mines, whe

Query 18: Print the  name, related terms and definitions of articles that have ABACISCUS in their name.

In [39]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
    PREFIX eb: <https://w3id.org/eb#>
       SELECT * WHERE {
       ?article a eb:Article .
       ?article eb:relatedTerms ?relatedTerms . 
       ?article eb:name ?name .
       ?article eb:definition ?definition.
  	   FILTER regex(?name, "^ABACISCUS") 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("URI: %s -- Name %s -- Related Term: %s -- Definition %s " %(result["article"]["value"], result["name"]["value"], result["relatedTerms"]["value"], result["definition"]["value"] ))
    print("----")

URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS -- Related Term: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_1 -- Definition See ABACUS. 
----
URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS -- Related Term: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_0 -- Definition See ABACUS. 
----
URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS -- Related Term: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_2 -- Definition See ABACUS. 
----
URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS -- Related Term: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_3 -- Definition See ABACUS. 
----
URI: https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACISCUS_0 -- Name ABACISCUS -- Related Term: https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACUS_1 -- De

Query 19:  Print URI and Term name of articles that have ABACISCUS in their names - Limited to 10

In [40]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?article a eb:Article .
       ?article eb:name ?name .
       FILTER regex(?name, "^ABACISCUS")
    }
    ORDER BY ?name
    LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("URI: %s -- Name %s " %(result["article"]["value"], result["name"]["value"]))
    print("----")

URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS 
----
URI: https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACISCUS_0 -- Name ABACISCUS 
----
URI: https://w3id.org/eb/i/Article/9929777383804340_193322698_ABACISCUS_0 -- Name ABACISCUS 
----


Query 20:  Print the year of an edition

In [43]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?year WHERE {
       ?edition a eb:Edition .
       ?edition eb:publicationYear ?year 
    }

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [44]:
results["results"]["bindings"][0]["year"]["value"]


'1778'

Query 22: Print the name of an Editor

In [46]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
         PREFIX eb: <https://w3id.org/eb#>
         SELECT DISTINCT ?name
         WHERE {
         ?instance eb:editor ?Editor.
          ?Editor eb:name ?name .
       }
      
    """)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["name"]["value"]

'Smellie, William'

Query 23: Count Query

In [47]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?a) as ?count)
    WHERE {
    	?a ?b ?d
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'188840'

Query 24: Count query

In [48]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT ( ?a) as ?count)
    WHERE {
    	?a ?b ?d
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'1638239'

Query 25: Print 10 names of Articles or Topics resources

In [49]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?a
    WHERE {{
    	?b a eb:Article .
    	?b eb:name ?a}
  		UNION {
    	?b a eb:Topic .
    	?b eb:name ?a}
}
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["a"]["value"])

MATRICES
AMPHILOCHIUS
ISATIS
BAG
VIVES
GUINEA
SUCCINIC
MAMALUKES
DIONYSIA
WETLUGA


Query 26: Print 10 definitions of articles which contain Scotland and Glasgow, and optional Mull.

In [50]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?d
    WHERE {
    	?b a eb:Article .
    	?b eb:definition ?d
        FILTER (CONTAINS(?d, "Scotland")) 
        FILTER (CONTAINS(?d, "Glasgow")) 
        OPTIONAL{FILTER CONTAINS(?d, "Mull") }
} LIMIT 10

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["d"]["value"])
    print("----")

a town of Scotland, in the county of Renfrew, six miles west of Glasgow.
----
a county of Scotland, lying westward of Glasgow, and comprehending the countries of Lorn, Cowal, Knapdale, Kintyfe, together with the islands Mull, Jura, Ifla, foe. It gives the title of duke to the noble family of Campbell.
----
a town of Scotland, in Clydefdale, seated on the river Clyde, eleven miles so-uth east of Glasgow; from whence the noble family of Hamiltontake their name, and title of duke. The town is seated in the middle of a very agreeable plain; and on the west of the town this family has a large park, which is near (even miles in circumference, inelofed with a high wall, full of deer and other game. The rivulet called Avon runs through the park, and falls into the river Clyde,over which last there is a bridge of free-stone. W. Long. 3. 50. N. Lat. 55. 40. The original name of this place, or the lands about it, was Cadzow or Cadyotv, a barony granted to an ancestor of the noble owner, on the fo

Query 27:  Print definition of Articles resources with the name PAISLEY

In [51]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?y WHERE {
       ?article a eb:Article ;
                eb:name "PAISLEY" ;
                eb:definition ?definition .
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?y
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["definition"]["value"], r["y"]["value"])

a town of Scotland, in the county of Renfrew, six miles west of Glasgow. 1773
a town of Scotland, in the county of Renfrew, six miles west of Glasgow. 1771
a town of Renfrewfliire, in Scotland, situated about six miles west of Glasgow, on ibe river White-Cart, ever which there are two Honebridges of two arches each, and one which confiftsof three arches. The town is very ancient ; but was of much less cosfequence formerly than it is at present. The old part of it runs from ea H to wed upon the south Hope of a ridge of hills, from which there is a fine profped of the city of Glasgow and the adjacent country ; but to the southward, the view terminates in a ridge of green bills, about two miles diRant. Including the late buildings and suburbs, it is fully a mile long, and nearly as much in breadth. On the ea H side of the river Cart, Hand the abbey and new town; which la R was lately feued off by the Earl of Abcrcorn, and already conli Rs of a number of handsome buildings. The streets are

Query 28: Print definition and year of articles with the name ABACUS. If those articles have related terms, print those ones as well. 

In [52]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?y  ?r WHERE {
       ?article a eb:Article ;
                eb:name "ABACUS" ;
                eb:definition ?definition ;
                OPTIONAL {?article eb:relatedTerms ?r.}
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?y
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print("DEF: %s ----|| --- Year: %s "% (r["definition"]["value"], r["y"]["value"]))
    print("---")

DEF: among the ancients, was a kind of cup Livy, deferibing the luxury into which the Romans degenerated after the conquest of Alia, says they had their abaci, beds, &c. plated over ----|| --- Year: 1815 
---
DEF: among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says they had their abaci, beds, &c. plated over with gold. ----|| --- Year: 1853 
---
DEF: among the ancients, was a kind of cupboard, or buffet. J Livy, deferibing the luxury into which the Romans degenerated after the conquest of Asia, says, They had their abaci, beds, &c. plated over with gold. (Dec. IV. Lib. ix.) ----|| --- Year: 1778 
---
DEF: in architecture, II. 5; III. 464 —cupboard, II. 5—arithmetical instrument, ib. ----|| --- Year: 1842 
---
DEF: among the ancients, was a kind of cupboard or buffet. Livy, describing the luxury into which the Romans degenerated after the conquest of Asia, says they had their abaci, beds, &c

Query 29: Print the years of articles which have ABACUS in their names

In [53]:
term="ABACUS"
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?article  ?year WHERE {
       ?article a eb:Article .
       ?article eb:name "ABACUS"  .
       ?article eb:definition ?definition .
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?year

    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["year"]["value"])

1815
1853
1778
1842
1823
1797
1842
1771
1823
1797
1797
1771
1773
1823
1773
1773
1842
1853
1823
1815
1778
1771
1771
1773
1842


Query 30: Print the number of volumes of the Edition with the uri https://w3id.org/eb/i/Edition/992277653804341

In [57]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Edition/992277653804341>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?v) as ?count)
    WHERE {
        %s eb:hasPart ?v.
    	?v ?b ?c
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'3'

Query 31: Print all Editions uris

In [59]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?e a eb:Edition .
    }
""" 

print(query)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results


PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?e a eb:Edition .
    }



{'head': {'vars': ['e']},
 'results': {'bindings': [{'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/997902523804341'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/997902543804341'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9929777383804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9910796273804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9922270543804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/992277653804341'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9910796253804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9910796233804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9910796343804340'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9929192893804340'}}]}}

Query 32: Print Edition URIS and their years

In [61]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?enum ?e ?y WHERE {
       ?e a eb:Edition ;
            eb:number ?enum ;
            eb:publicationYear ?y.
               
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["enum"]["value"], i["e"]["value"], i["y"]["value"])

2 https://w3id.org/eb/i/Edition/997902523804341 1778
3 https://w3id.org/eb/i/Edition/997902543804341 1797
8 https://w3id.org/eb/i/Edition/9929777383804340 1853
7 https://w3id.org/eb/i/Edition/9910796273804340 1842
5 https://w3id.org/eb/i/Edition/9922270543804340 1815
1 https://w3id.org/eb/i/Edition/992277653804341 1771
6 https://w3id.org/eb/i/Edition/9910796253804340 1823
4 https://w3id.org/eb/i/Edition/9910796233804340 1810
0 https://w3id.org/eb/i/Edition/9910796343804340 1801
1 https://w3id.org/eb/i/Edition/9929192893804340 1773


Query 33: Print the volumes of uris, number and letters for the Edition with URI https://w3id.org/eb/i/Edition/992277653804341

In [73]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Edition/992277653804341>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?v ?vnum ?part ?letters WHERE {
       %s eb:hasPart ?v .
       ?v eb:number ?vnum ; 
          eb:letters ?letters .
          OPTIONAL {?v eb:part ?part; }
     
               
} 
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    if "part" in i:
        print(i["v"]["value"], i["vnum"]["value"],i["letters"]["value"], i["part"]["value"])
    else:
        print(i["v"]["value"], i["vnum"]["value"], i["letters"]["value"])

https://w3id.org/eb/i/Volume/992277653804341_144133902 2 C-L
https://w3id.org/eb/i/Volume/992277653804341_144133903 3 M-Z
https://w3id.org/eb/i/Volume/992277653804341_144133901 1 A-B


Query 35: Given the volume with URI https://w3id.org/eb/i/Volume/992277653804341_144133901, print its number, title, part, metsxml, volume Id, permanent URL, number of Pages and letters. 

In [63]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?num ?title ?part ?metsXML ?volumeId ?permanentURL ?numberOfPages ?letters WHERE {
       %s eb:number ?num ;
          eb:title ?title;
          eb:metsXML ?metsXML;
          eb:volumeId ?volumeId;
          eb:permanentURL ?permanentURL;
          eb:numberOfPages ?numberOfPages;
          eb:letters ?letters.
       OPTIONAL {%s eb:part ?part. }
      
               
}
""" % (uri, uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i)

{'num': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1'}, 'title': {'type': 'literal', 'value': 'Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan'}, 'metsXML': {'type': 'literal', 'value': '144133901-mets.xml'}, 'volumeId': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '144133901'}, 'permanentURL': {'type': 'literal', 'value': 'https://digital.nls.uk/144133901'}, 'numberOfPages': {'type': 'literal', 'value': '832'}, 'letters': {'type': 'literal', 'value': 'A-B'}}


Query 36: Count the number of terms for the Volume with Uri: https://w3id.org/eb/i/Volume/992277653804341_144133901

In [64]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (?t) as ?count)
    WHERE {
        %s eb:hasPart ?t .
        ?t ?b ?c 
        
} 
""" % (uri)
      
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'21354'

Query 37: Count the number of distintc terms for the Volume with Uri: https://w3id.org/eb/i/Volume/992277653804341_144133901

In [65]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?t) as ?count)
    WHERE {
        %s eb:hasPart ?t .
        ?t a eb:Article
        
} 
""" % (uri)
      
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'type': 'literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '2505'}}]}}

Query 38: Count query

In [66]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
       PREFIX eb: <https://w3id.org/eb#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT (count (DISTINCT ?a) as ?count)
       WHERE {
            %s eb:hasPart ?b .
            ?b a eb:Article .
            ?b eb:name ?a.
      }


""" % (uri)
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'2440'

Query 39: Give me 10 topics uris

In [70]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?topic ?y WHERE {
       ?topic a eb:Topic ;
    }
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["topic"]["value"])

https://w3id.org/eb/i/Topic/997902543804341_191253800_SIMSIMSIM_0
https://w3id.org/eb/i/Topic/9910796273804340_194474782_NEWBRUNSWICK_0
https://w3id.org/eb/i/Topic/9929777383804340_193469393_SHIELD_0
https://w3id.org/eb/i/Topic/992277653804341_144133901_ALGEBRA_1
https://w3id.org/eb/i/Topic/9910796233804340_191678901_TO_2
https://w3id.org/eb/i/Topic/9910796273804340_193322688_DOR_2
https://w3id.org/eb/i/Topic/9910796253804340_192547783_PHYSIOGNOMY_0
https://w3id.org/eb/i/Topic/9922270543804340_192984256_SCOSCODMS_0
https://w3id.org/eb/i/Topic/9910796253804340_191689065_MUSIC_0
https://w3id.org/eb/i/Topic/9910796273804340_193322690_CAPILLARYACTION_0


Query 40: Give me the volume number and letter for the volume with the uri: https://w3id.org/eb/i/Volume/992277653804341_144133902

In [71]:
sparql = SPARQLWrapper("http://localhost:3030/total_eb/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133902>"
query="""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?vnum ?letters ?part WHERE {
       %s eb:number ?vnum ;
          eb:letters ?letters .
           OPTIONAL {%s eb:part ?part; }
       
    
    } """ % (uri, uri)
sparql.setQuery(query)         
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=results["results"]["bindings"][0]


if "part" in i:
    data= i["vnum"]["value"]+ " "+ i["letters"]["value"]+ "Part "+ i["part"]["value"]
else:
    data= i["vnum"]["value"]+ " "+ i["letters"]["value"]

data

'2 C-L'